In [ ]:
!pip install rasgoql~=1.0.0
!pip install rasgotransforms~=1.0.0

!pip show rasgoql
!Python -V

In [2]:
import os
import pandas as pd
import rasgoql
import numpy as np

In [3]:
env_file = "<path_to_env_file>"
creds = rasgoql.SnowflakeCredentials.from_env(env_file)

In [4]:
rql = rasgoql.connect(creds)

# Set Rasgo verbose Debug output to true so we can can take a closer look at what's happening 
rql.set_verbose(True)

# Explore available datasets
rql.list_tables('ADVENTUREWORKS')

INFO:Snowflake DataWarehouse:Executing query: SELECT TABLE_NAME, TABLE_CATALOG||'.'||TABLE_SCHEMA||'.'||TABLE_NAME AS FQTN, CASE TABLE_TYPE WHEN 'BASE TABLE' THEN 'TABLE' ELSE TABLE_TYPE END AS TABLE_TYPE, ROW_COUNT, CREATED, LAST_ALTERED  FROM ADVENTUREWORKS.INFORMATION_SCHEMA.TABLES 


,TABLE_NAME,FQTN,TABLE_TYPE,ROW_COUNT,CREATED,LAST_ALTERED
0,ADVENTUREWORKSDWBUILDVERSION,ADVENTUREWORKS.PUBLIC.ADVENTUREWORKSDWBUILDVER...,TABLE,0,2021-11-06 09:07:48.039000-07:00,2022-02-20 00:00:32.444000-08:00
1,DIMACCOUNT,ADVENTUREWORKS.PUBLIC.DIMACCOUNT,TABLE,99,2021-11-06 09:07:48.012000-07:00,2022-02-20 00:00:32.444000-08:00
2,DIMCURRENCY,ADVENTUREWORKS.PUBLIC.DIMCURRENCY,TABLE,105,2021-11-06 09:07:48.071000-07:00,2022-02-20 00:00:32.441000-08:00
3,DIMCUSTOMER,ADVENTUREWORKS.PUBLIC.DIMCUSTOMER,TABLE,18484,2021-11-06 09:07:47.965000-07:00,2022-02-20 00:00:32.441000-08:00
4,DIMDATE,ADVENTUREWORKS.PUBLIC.DIMDATE,TABLE,3652,2021-11-06 09:07:47.965000-07:00,2022-02-20 00:00:32.441000-08:00
5,DIMDEPARTMENTGROUP,ADVENTUREWORKS.PUBLIC.DIMDEPARTMENTGROUP,TABLE,7,2021-11-06 09:07:48.071000-07:00,2022-02-20 00:00:32.441000-08:00
6,DIMEMPLOYEE,ADVENTUREWORKS.PUBLIC.DIMEMPLOYEE,TABLE,296,2021-11-06 09:07:47.965000-07:00,2022-02-20 00:00:32.444000-08:00
7,DIMGEOGRAPHY,ADVENTUREWORKS.PUBLIC.DIMGEOGRAPHY,TABLE,655,2021-11-06 09:07:48.037000-07:00,2022-02-20 00:00:32.444000-08:00
8,DIMORGANIZATION,ADVENTUREWORKS.PUBLIC.DIMORGANIZATION,TABLE,14,2021-11-06 09:07:48.115000-07:00,2022-02-20 00:00:32.444000-08:00
9,DIMPRODUCT,ADVENTUREWORKS.PUBLIC.DIMPRODUCT,TABLE,606,2021-11-06 09:07:47.965000-07:00,2022-02-20 00:00:32.441000-08:00


# Gather relevant data
In order to calculate the total net revenue for all our bikes sold via our online store, we need to combine  the individual order information with any promotion data for a given sale. In the ADVENTUREWORKS Dataset that information is spread across two tables: 'FACTINTERNETSALES' and 'DIMPROMOTION'. 

In [5]:
# Load Data directly from the data warehouse
sales_rql = rql.dataset('ADVENTUREWORKS.PUBLIC.FACTINTERNETSALES')
promotion_rql = rql.dataset('ADVENTUREWORKS.PUBLIC.DIMPROMOTION')

# Or Load it in as a CSV
sales_pd = pd.read_csv('https://github.com/rasgointelligence/RasgoQL/raw/7f238a6f1f6bc60d83ecbe5050492aaaf3cb9aab/tutorials/FactInternetSales.csv', 
                        names=['PRODUCTKEY', 'ORDERDATEKEY', 'DUEDATEKEY', 'SHIPDATEKEY',
                               'CUSTOMERKEY', 'PROMOTIONKEY', 'CURRENCYKEY', 'SALESTERRITORYKEY',
                               'SALESORDERNUMBER', 'SALESORDERLINENUMBER', 'REVISIONNUMBER',
                               'ORDERQUANTITY', 'UNITPRICE', 'EXTENDEDAMOUNT', 'UNITPRICEDISCOUNTPCT',
                               'DISCOUNTAMOUNT', 'PRODUCTSTANDARDCOST', 'TOTALPRODUCTCOST',
                               'SALESAMOUNT', 'TAXAMT', 'FREIGHT', 'CARRIERTRACKINGNUMBER',
                               'CUSTOMERPONUMBER', 'ORDERDATE', 'DUEDATE', 'SHIPDATE'], 
                        sep='|',
                        parse_dates=True,
                        encoding='utf_16_le'
                       )

promotion_pd = pd.read_csv('https://raw.githubusercontent.com/rasgointelligence/RasgoQL/7f238a6f1f6bc60d83ecbe5050492aaaf3cb9aab/tutorials/DimPromotion.csv',
                           names=['PROMOTIONKEY', 'PROMOTIONALTERNATEKEY', 'ENGLISHPROMOTIONNAME',
                                  'SPANISHPROMOTIONNAME', 'FRENCHPROMOTIONNAME', 'DISCOUNTPCT',
                                  'ENGLISHPROMOTIONTYPE', 'SPANISHPROMOTIONTYPE', 'FRENCHPROMOTIONTYPE',
                                  'ENGLISHPROMOTIONCATEGORY', 'SPANISHPROMOTIONCATEGORY',
                                  'FRENCHPROMOTIONCATEGORY', 'STARTDATE', 'ENDDATE', 'MINQTY', 'MAXQTY'],
                           sep='|',
                           parse_dates=True,
                           encoding='utf_16_le'
                           )

display(sales_pd)
display(promotion_pd)

INFO:Snowflake DataWarehouse:Executing query: SHOW OBJECTS LIKE 'FACTINTERNETSALES' IN ADVENTUREWORKS.PUBLIC
INFO:Snowflake DataWarehouse:Executing query: SHOW OBJECTS LIKE 'DIMPROMOTION' IN ADVENTUREWORKS.PUBLIC


,PRODUCTKEY,ORDERDATEKEY,DUEDATEKEY,SHIPDATEKEY,CUSTOMERKEY,PROMOTIONKEY,CURRENCYKEY,SALESTERRITORYKEY,SALESORDERNUMBER,SALESORDERLINENUMBER,...,PRODUCTSTANDARDCOST,TOTALPRODUCTCOST,SALESAMOUNT,TAXAMT,FREIGHT,CARRIERTRACKINGNUMBER,CUSTOMERPONUMBER,ORDERDATE,DUEDATE,SHIPDATE
0,310,20101229,20110110,20110105,21768,1,19,6,SO43697,1,...,2171.2942,2171.2942,3578.2700,286.2616,89.4568,NaN,NaN,2010-12-29 00:00:00,2011-01-10 00:00:00,2011-01-05 00:00:00
1,346,20101229,20110110,20110105,28389,1,39,7,SO43698,1,...,1912.1544,1912.1544,3399.9900,271.9992,84.9998,NaN,NaN,2010-12-29 00:00:00,2011-01-10 00:00:00,2011-01-05 00:00:00
2,346,20101229,20110110,20110105,25863,1,100,1,SO43699,1,...,1912.1544,1912.1544,3399.9900,271.9992,84.9998,NaN,NaN,2010-12-29 00:00:00,2011-01-10 00:00:00,2011-01-05 00:00:00
3,336,20101229,20110110,20110105,14501,1,100,4,SO43700,1,...,413.1463,413.1463,699.0982,55.9279,17.4775,NaN,NaN,2010-12-29 00:00:00,2011-01-10 00:00:00,2011-01-05 00:00:00
4,346,20101229,20110110,20110105,11003,1,6,9,SO43701,1,...,1912.1544,1912.1544,3399.9900,271.9992,84.9998,NaN,NaN,2010-12-29 00:00:00,2011-01-10 00:00:00,2011-01-05 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60393,485,20140128,20140209,20140204,15868,1,100,6,SO75122,1,...,8.2205,8.2205,21.9800,1.7584,0.5495,NaN,NaN,2014-01-28 00:00:00,2014-02-09 00:00:00,2014-02-04 00:00:00
60394,225,20140128,20140209,20140204,15868,1,100,6,SO75122,2,...,6.9223,6.9223,8.9900,0.7192,0.2248,NaN,NaN,2014-01-28 00:00:00,2014-02-09 00:00:00,2014-02-04 00:00:00
60395,485,20140128,20140209,20140204,18759,1,100,6,SO75123,1,...,8.2205,8.2205,21.9800,1.7584,0.5495,NaN,NaN,2014-01-28 00:00:00,2014-02-09 00:00:00,2014-02-04 00:00:00
60396,486,20140128,20140209,20140204,18759,1,100,6,SO75123,2,...,59.4660,59.4660,159.0000,12.7200,3.9750,NaN,NaN,2014-01-28 00:00:00,2014-02-09 00:00:00,2014-02-04 00:00:00


,PROMOTIONKEY,PROMOTIONALTERNATEKEY,ENGLISHPROMOTIONNAME,SPANISHPROMOTIONNAME,FRENCHPROMOTIONNAME,DISCOUNTPCT,ENGLISHPROMOTIONTYPE,SPANISHPROMOTIONTYPE,FRENCHPROMOTIONTYPE,ENGLISHPROMOTIONCATEGORY,SPANISHPROMOTIONCATEGORY,FRENCHPROMOTIONCATEGORY,STARTDATE,ENDDATE,MINQTY,MAXQTY
0,1,1,No Discount,Sin descuento,Aucune remise,0.00,No Discount,Sin descuento,Aucune remise,No Discount,Sin descuento,Aucune remise,2010-11-29 00:00:00,2014-06-30 00:00:00,0,NaN
1,2,2,Volume Discount 11 to 14,Descuento por volumen (entre 11 y 14),Remise sur quantité (de 11 à 14),0.02,Volume Discount,Descuento por volumen,Remise sur quantité,Reseller,Distribuidor,Revendeur,2010-12-29 00:00:00,2013-12-28 00:00:00,11,14.0
2,3,3,Volume Discount 15 to 24,Descuento por volumen (entre 15 y 24),Remise sur quantité (de 15 à 24),0.05,Volume Discount,Descuento por volumen,Remise sur quantité,Reseller,Distribuidor,Revendeur,2010-12-29 00:00:00,2013-12-28 00:00:00,15,24.0
3,4,4,Volume Discount 25 to 40,Descuento por volumen (entre 25 y 40),Remise sur quantité (de 25 à 40),0.10,Volume Discount,Descuento por volumen,Remise sur quantité,Reseller,Distribuidor,Revendeur,2010-12-29 00:00:00,2013-12-28 00:00:00,25,40.0
4,5,5,Volume Discount 41 to 60,Descuento por volumen (entre 41 y 60),Remise sur quantité (de 41 à 60),0.15,Volume Discount,Descuento por volumen,Remise sur quantité,Reseller,Distribuidor,Revendeur,2010-12-29 00:00:00,2013-12-28 00:00:00,41,60.0
5,6,6,Volume Discount over 60,Descuento por volumen (más de 60),Remise sur quantité (au-delà de 60),0.20,Volume Discount,Descuento por volumen,Remise sur quantité,Reseller,Distribuidor,Revendeur,2010-12-29 00:00:00,2013-12-28 00:00:00,61,NaN
6,7,7,Mountain-100 Clearance Sale,"Liquidación de bicicleta de montaña, 100",Liquidation VTT 100,0.35,Discontinued Product,Descatalogado,Ce produit n'est plus commercialisé,Reseller,Distribuidor,Revendeur,2011-11-12 00:00:00,2011-12-28 00:00:00,0,NaN
7,8,8,Sport Helmet Discount-2002,"Casco deportivo, descuento: 2002",Remise sur les casques sport - 2002,0.10,Seasonal Discount,Descuento de temporada,Remise saisonnière,Reseller,Distribuidor,Revendeur,2011-12-29 00:00:00,2012-01-28 00:00:00,0,NaN
8,9,9,Road-650 Overstock,"Bicicleta de carretera: 650, oferta especial",Déstockage Vélo de route 650,0.30,Excess Inventory,Inventario excedente,Déstockage,Reseller,Distribuidor,Revendeur,2011-12-29 00:00:00,2012-02-28 00:00:00,0,NaN
9,10,10,Mountain Tire Sale,Oferta de cubierta de montaña,Vente de pneus de VTT,0.50,Excess Inventory,Inventario excedente,Déstockage,Customer,Cliente,Client,2012-12-12 00:00:00,2013-02-26 00:00:00,0,NaN


# Pivot data by date
Starting with Pandas, lets pivot the local csv containing sales data to group things by date.

In [6]:
gross_sales = pd.pivot_table(sales_pd,
                             index='ORDERDATE',
                             aggfunc= sum)
gross_sales

,CARRIERTRACKINGNUMBER,CURRENCYKEY,CUSTOMERKEY,CUSTOMERPONUMBER,DISCOUNTAMOUNT,DUEDATEKEY,EXTENDEDAMOUNT,FREIGHT,ORDERDATEKEY,ORDERQUANTITY,...,PROMOTIONKEY,REVISIONNUMBER,SALESAMOUNT,SALESORDERLINENUMBER,SALESTERRITORYKEY,SHIPDATEKEY,TAXAMT,TOTALPRODUCTCOST,UNITPRICE,UNITPRICEDISCOUNTPCT
ORDERDATE,,,,,,,,,,,,,,,,,,,,,
2010-12-29 00:00:00,0.0,264,101524,0.0,0,100550550,14477.3382,361.9337,100506145,5,...,5,5,14477.3382,5,27,100550525,1158.1871,8320.9037,14477.3382,0
2010-12-30 00:00:00,0.0,118,66285,0.0,0,80440444,13931.5200,348.2882,80404920,4,...,4,4,13931.5200,4,31,80440424,1114.5216,8152.8372,13931.5200,0
2010-12-31 00:00:00,0.0,310,108147,0.0,0,100550560,15012.1782,375.3047,100506155,5,...,5,5,15012.1782,5,36,100550535,1200.9743,9098.3231,15012.1782,0
2011-01-01 00:00:00,0.0,129,41119,0.0,0,40220226,7156.5400,178.9136,40220202,2,...,2,2,7156.5400,2,9,40220216,572.5232,4342.5884,7156.5400,0
2011-01-02 00:00:00,0.0,216,99453,0.0,0,100550570,15012.1782,375.3047,100550510,5,...,5,5,15012.1782,5,41,100550545,1200.9743,9098.3231,15012.1782,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2014-01-24 00:00:00,0.0,6500,1298903,0.0,0,1309113325,1502.8500,37.5742,1309108060,65,...,65,65,1502.8500,117,374,1309108515,120.2280,632.1362,1502.8500,0
2014-01-25 00:00:00,0.0,8200,1541874,0.0,0,1651496892,1747.6700,43.6954,1651490250,82,...,82,82,1747.6700,166,538,1651496482,139.8136,782.6954,1747.6700,0
2014-01-26 00:00:00,0.0,6800,1232559,0.0,0,1369534076,1847.4600,46.1895,1369528568,68,...,68,68,1847.4600,122,420,1369533736,147.7968,849.7083,1847.4600,0


Before we join the promotion data in, lets isolate things to only the data that's necessary to calculate net revenue. Here we just create a new dataframe only containing the promotion key information and the discount percentage.

In [7]:
lean_promo = pd.DataFrame(data=promotion_pd,
                            columns=['PROMOTIONKEY', 'DISCOUNTPCT']
                            )
lean_promo

,PROMOTIONKEY,DISCOUNTPCT
0,1,0.00
1,2,0.02
2,3,0.05
3,4,0.10
4,5,0.15
5,6,0.20
6,7,0.35
7,8,0.10
8,9,0.30
9,10,0.50


In [8]:
# Merge the two datasets together
sales_with_promo = pd.merge(sales_pd, lean_promo, left_on='PROMOTIONKEY', right_index=True, how='left')

sales_with_promo

,PRODUCTKEY,ORDERDATEKEY,DUEDATEKEY,SHIPDATEKEY,CUSTOMERKEY,PROMOTIONKEY_x,CURRENCYKEY,SALESTERRITORYKEY,SALESORDERNUMBER,SALESORDERLINENUMBER,...,SALESAMOUNT,TAXAMT,FREIGHT,CARRIERTRACKINGNUMBER,CUSTOMERPONUMBER,ORDERDATE,DUEDATE,SHIPDATE,PROMOTIONKEY_y,DISCOUNTPCT
0,310,20101229,20110110,20110105,21768,1,19,6,SO43697,1,...,3578.2700,286.2616,89.4568,NaN,NaN,2010-12-29 00:00:00,2011-01-10 00:00:00,2011-01-05 00:00:00,2,0.02
1,346,20101229,20110110,20110105,28389,1,39,7,SO43698,1,...,3399.9900,271.9992,84.9998,NaN,NaN,2010-12-29 00:00:00,2011-01-10 00:00:00,2011-01-05 00:00:00,2,0.02
2,346,20101229,20110110,20110105,25863,1,100,1,SO43699,1,...,3399.9900,271.9992,84.9998,NaN,NaN,2010-12-29 00:00:00,2011-01-10 00:00:00,2011-01-05 00:00:00,2,0.02
3,336,20101229,20110110,20110105,14501,1,100,4,SO43700,1,...,699.0982,55.9279,17.4775,NaN,NaN,2010-12-29 00:00:00,2011-01-10 00:00:00,2011-01-05 00:00:00,2,0.02
4,346,20101229,20110110,20110105,11003,1,6,9,SO43701,1,...,3399.9900,271.9992,84.9998,NaN,NaN,2010-12-29 00:00:00,2011-01-10 00:00:00,2011-01-05 00:00:00,2,0.02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60393,485,20140128,20140209,20140204,15868,1,100,6,SO75122,1,...,21.9800,1.7584,0.5495,NaN,NaN,2014-01-28 00:00:00,2014-02-09 00:00:00,2014-02-04 00:00:00,2,0.02
60394,225,20140128,20140209,20140204,15868,1,100,6,SO75122,2,...,8.9900,0.7192,0.2248,NaN,NaN,2014-01-28 00:00:00,2014-02-09 00:00:00,2014-02-04 00:00:00,2,0.02
60395,485,20140128,20140209,20140204,18759,1,100,6,SO75123,1,...,21.9800,1.7584,0.5495,NaN,NaN,2014-01-28 00:00:00,2014-02-09 00:00:00,2014-02-04 00:00:00,2,0.02
60396,486,20140128,20140209,20140204,18759,1,100,6,SO75123,2,...,159.0000,12.7200,3.9750,NaN,NaN,2014-01-28 00:00:00,2014-02-09 00:00:00,2014-02-04 00:00:00,2,0.02


In [9]:
# or 
sales_pd.merge(lean_promo, left_on='PROMOTIONKEY', right_index=True, how='left')

,PRODUCTKEY,ORDERDATEKEY,DUEDATEKEY,SHIPDATEKEY,CUSTOMERKEY,PROMOTIONKEY_x,CURRENCYKEY,SALESTERRITORYKEY,SALESORDERNUMBER,SALESORDERLINENUMBER,...,SALESAMOUNT,TAXAMT,FREIGHT,CARRIERTRACKINGNUMBER,CUSTOMERPONUMBER,ORDERDATE,DUEDATE,SHIPDATE,PROMOTIONKEY_y,DISCOUNTPCT
0,310,20101229,20110110,20110105,21768,1,19,6,SO43697,1,...,3578.2700,286.2616,89.4568,NaN,NaN,2010-12-29 00:00:00,2011-01-10 00:00:00,2011-01-05 00:00:00,2,0.02
1,346,20101229,20110110,20110105,28389,1,39,7,SO43698,1,...,3399.9900,271.9992,84.9998,NaN,NaN,2010-12-29 00:00:00,2011-01-10 00:00:00,2011-01-05 00:00:00,2,0.02
2,346,20101229,20110110,20110105,25863,1,100,1,SO43699,1,...,3399.9900,271.9992,84.9998,NaN,NaN,2010-12-29 00:00:00,2011-01-10 00:00:00,2011-01-05 00:00:00,2,0.02
3,336,20101229,20110110,20110105,14501,1,100,4,SO43700,1,...,699.0982,55.9279,17.4775,NaN,NaN,2010-12-29 00:00:00,2011-01-10 00:00:00,2011-01-05 00:00:00,2,0.02
4,346,20101229,20110110,20110105,11003,1,6,9,SO43701,1,...,3399.9900,271.9992,84.9998,NaN,NaN,2010-12-29 00:00:00,2011-01-10 00:00:00,2011-01-05 00:00:00,2,0.02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60393,485,20140128,20140209,20140204,15868,1,100,6,SO75122,1,...,21.9800,1.7584,0.5495,NaN,NaN,2014-01-28 00:00:00,2014-02-09 00:00:00,2014-02-04 00:00:00,2,0.02
60394,225,20140128,20140209,20140204,15868,1,100,6,SO75122,2,...,8.9900,0.7192,0.2248,NaN,NaN,2014-01-28 00:00:00,2014-02-09 00:00:00,2014-02-04 00:00:00,2,0.02
60395,485,20140128,20140209,20140204,18759,1,100,6,SO75123,1,...,21.9800,1.7584,0.5495,NaN,NaN,2014-01-28 00:00:00,2014-02-09 00:00:00,2014-02-04 00:00:00,2,0.02
60396,486,20140128,20140209,20140204,18759,1,100,6,SO75123,2,...,159.0000,12.7200,3.9750,NaN,NaN,2014-01-28 00:00:00,2014-02-09 00:00:00,2014-02-04 00:00:00,2,0.02


## Calculate Net Revenue

In [10]:
# Subtract DISCOUNT_PCT from SALESAMOUNT
sales_with_promo['SALESAMOUNT_NET'] = sales_with_promo['SALESAMOUNT'] - (sales_with_promo['SALESAMOUNT'] * sales_with_promo['DISCOUNTPCT'])
sales_with_promo

,PRODUCTKEY,ORDERDATEKEY,DUEDATEKEY,SHIPDATEKEY,CUSTOMERKEY,PROMOTIONKEY_x,CURRENCYKEY,SALESTERRITORYKEY,SALESORDERNUMBER,SALESORDERLINENUMBER,...,TAXAMT,FREIGHT,CARRIERTRACKINGNUMBER,CUSTOMERPONUMBER,ORDERDATE,DUEDATE,SHIPDATE,PROMOTIONKEY_y,DISCOUNTPCT,SALESAMOUNT_NET
0,310,20101229,20110110,20110105,21768,1,19,6,SO43697,1,...,286.2616,89.4568,NaN,NaN,2010-12-29 00:00:00,2011-01-10 00:00:00,2011-01-05 00:00:00,2,0.02,3506.704600
1,346,20101229,20110110,20110105,28389,1,39,7,SO43698,1,...,271.9992,84.9998,NaN,NaN,2010-12-29 00:00:00,2011-01-10 00:00:00,2011-01-05 00:00:00,2,0.02,3331.990200
2,346,20101229,20110110,20110105,25863,1,100,1,SO43699,1,...,271.9992,84.9998,NaN,NaN,2010-12-29 00:00:00,2011-01-10 00:00:00,2011-01-05 00:00:00,2,0.02,3331.990200
3,336,20101229,20110110,20110105,14501,1,100,4,SO43700,1,...,55.9279,17.4775,NaN,NaN,2010-12-29 00:00:00,2011-01-10 00:00:00,2011-01-05 00:00:00,2,0.02,685.116236
4,346,20101229,20110110,20110105,11003,1,6,9,SO43701,1,...,271.9992,84.9998,NaN,NaN,2010-12-29 00:00:00,2011-01-10 00:00:00,2011-01-05 00:00:00,2,0.02,3331.990200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60393,485,20140128,20140209,20140204,15868,1,100,6,SO75122,1,...,1.7584,0.5495,NaN,NaN,2014-01-28 00:00:00,2014-02-09 00:00:00,2014-02-04 00:00:00,2,0.02,21.540400
60394,225,20140128,20140209,20140204,15868,1,100,6,SO75122,2,...,0.7192,0.2248,NaN,NaN,2014-01-28 00:00:00,2014-02-09 00:00:00,2014-02-04 00:00:00,2,0.02,8.810200
60395,485,20140128,20140209,20140204,18759,1,100,6,SO75123,1,...,1.7584,0.5495,NaN,NaN,2014-01-28 00:00:00,2014-02-09 00:00:00,2014-02-04 00:00:00,2,0.02,21.540400
60396,486,20140128,20140209,20140204,18759,1,100,6,SO75123,2,...,12.7200,3.9750,NaN,NaN,2014-01-28 00:00:00,2014-02-09 00:00:00,2014-02-04 00:00:00,2,0.02,155.820000


In [11]:
# Use Rasgo to generate the sql from our pandas frame
net_sales_ds = rql.dataset_from_df(
    df=sales_with_promo,
    table_name='FACTINTERNETSALES_NET',
    method='replace'
)

# Review the SQL
print(net_sales_ds.sql())

# Export to DBT
net_sales_ds.to_dbt()

INFO:Snowflake DataWarehouse:Executing query: SHOW OBJECTS LIKE 'FACTINTERNETSALES_NET' IN rasgoql.public
INFO:Snowflake DataWarehouse:Executing query: CREATE OR REPLACE TABLE rasgoql.public.FACTINTERNETSALES_NET (
PRODUCTKEY INTEGER,
  ORDERDATEKEY INTEGER,
  DUEDATEKEY INTEGER,
  SHIPDATEKEY INTEGER,
  CUSTOMERKEY INTEGER,
  PROMOTIONKEY_x INTEGER,
  CURRENCYKEY INTEGER,
  SALESTERRITORYKEY INTEGER,
  SALESORDERNUMBER TEXT,
  SALESORDERLINENUMBER INTEGER,
  REVISIONNUMBER INTEGER,
  ORDERQUANTITY INTEGER,
  UNITPRICE REAL,
  EXTENDEDAMOUNT REAL,
  UNITPRICEDISCOUNTPCT INTEGER,
  DISCOUNTAMOUNT INTEGER,
  PRODUCTSTANDARDCOST REAL,
  TOTALPRODUCTCOST REAL,
  SALESAMOUNT REAL,
  TAXAMT REAL,
  FREIGHT REAL,
  CARRIERTRACKINGNUMBER REAL,
  CUSTOMERPONUMBER REAL,
  ORDERDATE TEXT,
  DUEDATE TEXT,
  SHIPDATE TEXT,
  PROMOTIONKEY_y INTEGER,
  DISCOUNTPCT REAL,
  SALESAMOUNT_NET REAL
) COMMENT='rasgoql' 
INFO:Snowflake DataWarehouse:Executing query: SHOW OBJECTS LIKE 'FACTINTERNETSALES_NET' 

create or replace TABLE FACTINTERNETSALES_NET (
	PRODUCTKEY NUMBER(38,0),
	ORDERDATEKEY NUMBER(38,0),
	DUEDATEKEY NUMBER(38,0),
	SHIPDATEKEY NUMBER(38,0),
	CUSTOMERKEY NUMBER(38,0),
	PROMOTIONKEY_X NUMBER(38,0),
	CURRENCYKEY NUMBER(38,0),
	SALESTERRITORYKEY NUMBER(38,0),
	SALESORDERNUMBER VARCHAR(16777216),
	SALESORDERLINENUMBER NUMBER(38,0),
	REVISIONNUMBER NUMBER(38,0),
	ORDERQUANTITY NUMBER(38,0),
	UNITPRICE FLOAT,
	EXTENDEDAMOUNT FLOAT,
	UNITPRICEDISCOUNTPCT NUMBER(38,0),
	DISCOUNTAMOUNT NUMBER(38,0),
	PRODUCTSTANDARDCOST FLOAT,
	TOTALPRODUCTCOST FLOAT,
	SALESAMOUNT FLOAT,
	TAXAMT FLOAT,
	FREIGHT FLOAT,
	CARRIERTRACKINGNUMBER FLOAT,
	CUSTOMERPONUMBER FLOAT,
	ORDERDATE VARCHAR(16777216),
	DUEDATE VARCHAR(16777216),
	SHIPDATE VARCHAR(16777216),
	PROMOTIONKEY_Y NUMBER(38,0),
	DISCOUNTPCT FLOAT,
	SALESAMOUNT_NET FLOAT
)COMMENT='rasgoql'
;


AttributeError: 'Dataset' object has no attribute 'to_dbt'

# RASGOQL
Now lets repeat the same excersise with RasgoQL and this time have all the compute done directly in the cloud data warehouse.

In [ ]:
# Grab only what we need from promo

promo_data = promotion_rql.drop_columns(
    include_cols=['PROMOTIONKEY', 'DISCOUNTPCT']
)
promo_data.save()

In [ ]:
# Combine them into one dataset
sales_and_promo = sales_rql.join(join_table=promo_data.fqtn,
                                 join_type='RIGHT',
                                 join_columns={'PROMOTIONKEY':'PROMOTIONKEY'}
                                )
sales_and_promo.save()
sales_and_promo.preview()

Lastly, we'll use the RasgoQL `math` transform to subtract DISCOUNTPCT amount from SALESAMOUNT for all orders in the warehouse. 

In [ ]:
# Calculate net sales 

net_sales = sales_and_promo.math(math_ops=['SALESAMOUNT * DISCOUNTPCT', 'SALESAMOUNT - DISCOUNTTOTAL'],
                                 names=['DISCOUNTTOTAL', 'NET_SALE']
                                )
net_sales.save()
net_sales.preview()

In [ ]:
# Review SQL
print(net_sales.sql())

In [ ]:
# Export to DBT 
net_sales.to_dbt('./online_store_net_salesv2')